In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import random

# Load the SQuAD dataset
squad = load_dataset('squad')

# Initialize the tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Preprocess the data
def preprocess_function(examples):
    inputs = ["context: " + context + " question: " + question for context, question in zip(examples['context'], examples['question'])]
    targets = [answer['text'][0] for answer in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./question-answer-model")
tokenizer.save_pretrained("./question-answer-model")


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import random

# Load the fine-tuned model and tokenizer
model_name = "./question-answer-model"  # Path to your saved model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def generate_question(context, model, tokenizer, max_length=150):
    """
    Generate a question based on the given context using the fine-tuned model.
    
    Args:
    - context (str): The context from which to generate the question.
    - model: The fine-tuned T5 model.
    - tokenizer: The T5 tokenizer.
    
    Returns:
    - question (str): The generated question.
    - answer (str): The answer to the generated question.
    """
    input_text = f"context: {context} question:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    
    outputs = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the question and answer
    question, answer = generated_text.split('question:')[1].strip().split('answer:')
    return question.strip(), answer.strip()

def generate_distractors(context, correct_answer, model, tokenizer, num_distractors=3, max_length=50):
    """
    Generate distractors based on the context and correct answer using the fine-tuned model.
    
    Args:
    - context (str): The context from which to generate the distractors.
    - correct_answer (str): The correct answer.
    - model: The fine-tuned T5 model.
    - tokenizer: The T5 tokenizer.
    - num_distractors (int): Number of distractors to generate.
    
    Returns:
    - distractors (list): A list of distractor answers.
    """
    distractors = []
    for _ in range(num_distractors):
        input_text = f"context: {context} correct_answer: {correct_answer} distractor:"
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        
        outputs = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        distractor = generated_text.split('distractor:')[1].strip()
        distractors.append(distractor)
    
    return distractors

def generate_mcq(context, model, tokenizer, num_choices=4):
    """
    Generate a multiple choice question (MCQ) with distractors based on the given context.
    
    Args:
    - context (str): The context from which to generate the MCQ.
    - model: The fine-tuned T5 model.
    - tokenizer: The T5 tokenizer.
    - num_choices (int): The total number of choices (including the correct answer).
    
    Returns:
    - question (str): The generated question.
    - choices (list): A list of answer choices including the correct answer.
    - correct_answer (str): The correct answer.
    """
    # Generate the question and correct answer
    question, correct_answer = generate_question(context, model, tokenizer)
    
    # Generate distractors
    num_distractors = num_choices - 1
    distractors = generate_distractors(context, correct_answer, model, tokenizer, num_distractors)
    
    # Combine the correct answer with distractors and shuffle
    choices = distractors + [correct_answer]
    random.shuffle(choices)
    
    return question, choices, correct_answer

# Example usage
context = """The Battle of Hastings was fought on 14 October 1066 between the Norman-French army of William, the Duke of Normandy, and an English army under the Anglo-Saxon King Harold Godwinson. It took place at Senlac Hill, approximately 10 kilometers northwest of Hastings, close to the present-day town of Battle, East Sussex, and was a decisive Norman victory."""

question, choices, correct_answer = generate_mcq(context, model, tokenizer)

print("Question:", question)
print("Choices:", choices)
print("Correct Answer:", correct_answer)
